In [ ]:
import requests
import json

# Elasticsearch endpoint
es_url = "http://localhost:9200/products"

# Sample data file (replace with actual JSON data or load dynamically)
with open('products.json') as f:
    data = json.load(f)

# Create index with basic mapping (optional)
index_mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "product_id": {"type": "keyword"},
            "name": {"type": "text"},
            "price": {"type": "float"},
            "category": {"type": "keyword"}
        }
    }
}

# Check if index exists
response = requests.head(es_url)
if response.status_code == 404:  # Index does not exist
    create_index_response = requests.put(es_url, json=index_mapping)
    if create_index_response.status_code == 200:
        print("Index 'products' created successfully")
    else:
        print("Failed to create index:", create_index_response.text)

# Bulk ingest data
bulk_data = ""
for product in data:
    bulk_data += json.dumps({"index": {"_id": product["product_id"]}}) + "\n"
    bulk_data += json.dumps(product) + "\n"

bulk_response = requests.post(f"{es_url}/_bulk", headers={"Content-Type": "application/json"}, data=bulk_data)
if bulk_response.status_code == 200:
    print("Data ingested into Elasticsearch index 'products'")
else:
    print("Failed to ingest data:", bulk_response.text)


In [ ]:
from neo4j import GraphDatabase
import json

uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"  # replace with your Neo4j password

driver = GraphDatabase.driver(uri, auth=(user, password))

with open('products.json') as f:
    data = json.load(f)

def ingest_product(tx, product):
    tx.run("""
        MERGE (p:Product {id: $id})
        SET p.name = $name, p.category = $category, p.vendor = $vendor
    """, id=product['product_id'], name=product['name'], category=product['category'], vendor=product['vendor'])

    for subtopic in product.get('subtopics', []):
        tx.run("""
            MATCH (p:Product {id: $pid})
            MERGE (s:Subtopic {id: $sid})
            SET s.name = $sname
            MERGE (p)-[:HAS_SUBTOPIC]->(s)
        """, pid=product['product_id'], sid=subtopic['subtopic_id'], sname=subtopic['name'])

        for issue in subtopic.get('issues', []):
            tx.run("""
                MATCH (s:Subtopic {id: $sid})
                MERGE (i:Issue {id: $iid})
                SET i.title = $title, i.description = $desc
                MERGE (s)-[:HAS_ISSUE]->(i)
            """, sid=subtopic['subtopic_id'], iid=issue['issue_id'], title=issue['title'], desc=issue['description'])

            for cause in issue.get('root_causes', []):
                tx.run("""
                    MATCH (i:Issue {id: $iid})
                    MERGE (r:RootCause {id: $rcid})
                    SET r.description = $desc, r.probability = $prob
                    MERGE (i)-[:HAS_ROOT_CAUSE]->(r)
                """, iid=issue['issue_id'], rcid=cause['cause_id'], desc=cause['description'], prob=cause['probability'])

            for step in issue.get('solution_steps', []):
                tx.run("""
                    MATCH (i:Issue {id: $iid})
                    MERGE (ss:SolutionStep {id: $ssid})
                    SET ss.description = $desc, ss.order = $order
                    MERGE (i)-[:HAS_SOLUTION_STEP]->(ss)
                """, iid=issue['issue_id'], ssid=step['step_id'], desc=step['description'], order=step['order'])

            for article in issue.get('articles', []):
                tx.run("""
                    MATCH (i:Issue {id: $iid})
                    MERGE (a:Article {id: $aid})
                    SET a.title = $title, a.summary = $summary, a.url = $url
                    MERGE (i)-[:HAS_ARTICLE]->(a)
                """, iid=issue['issue_id'], aid=article['article_id'], title=article['title'], summary=article['summary'], url=article['url'])

with driver.session() as session:
    for product in data:
        session.write_transaction(ingest_product, product)

print("Data ingested into Neo4j")
driver.close()


In [ ]:
import json
import requests
from neo4j import GraphDatabase

# ======= LLM wrapper (your own function) =======
def generate_text(prompt):
    # Your LLM inference logic here
    raise NotImplementedError("Replace with your LLM call")

# ======= Elasticsearch Query via requests =======
def search_elasticsearch(query, top_k=3):
    body = {
        "query": {
            "match": {
                "content": {
                    "query": query
                }
            }
        },
        "size": top_k
    }
    res = requests.get(
        "http://localhost:9200/products/_search",
        headers={"Content-Type": "application/json"},
        data=json.dumps(body)
    )
    hits = res.json().get("hits", {}).get("hits", [])
    return [hit["_source"] for hit in hits]

# ======= Neo4j Cypher Runner =======
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_cypher_query(cypher_query):
    def fetch(tx):
        result = tx.run(cypher_query)
        return [record.data() for record in result]
    with driver.session() as session:
        return session.read_transaction(fetch)

# ======= LLM-powered Functions =======

def extract_query_details(user_query):
    prompt = f"""
You are an intelligent assistant that extracts key information from user queries related to IT issues.

User query: "{user_query}"

Extract the following as JSON:
- product
- subtopic (if any)
- intent/issue in plain text

Return only valid JSON.
"""
    response = generate_text(prompt)
    return json.loads(response)

def generate_cypher_query(details):
    prompt = f"""
You are a graph query generator. Given the following extracted info:

Product: {details['product']}
Subtopic: {details.get('subtopic')}
Issue description: {details['intent']}

Generate a Cypher query that:
- Matches the product node
- Traverses subtopics and issues
- Optionally matches root causes, solution steps, and articles
Return the full Cypher query only, no explanation.
"""
    return generate_text(prompt).strip()

def match_relevant_issue(user_query, candidate_issues):
    prompt = f"""
A user asked: "{user_query}"

Here are candidate issues from the knowledge graph:
{json.dumps(candidate_issues, indent=2)}

From this list, pick the best matching issue. Return a JSON:
- matched_issue_title
- why_it_matches (1-2 lines)
"""
    return json.loads(generate_text(prompt))

def generate_final_answer(user_query, kg_info, es_articles):
    prompt = f"""
You are a helpful IT assistant.

User query: "{user_query}"

Knowledge Graph Info:
- Issue: {kg_info.get('matched_issue_title')}
- Reasoning: {kg_info.get('why_it_matches')}

Self-help Articles:
{json.dumps(es_articles, indent=2)}

Use all the above to generate a helpful response to the user in clear language.
"""
    return generate_text(prompt)

# ======= Full Pipeline =======

def full_pipeline(user_query):
    details = extract_query_details(user_query)

    cypher = generate_cypher_query(details)
    candidate_issues = run_cypher_query(cypher)

    kg_match = match_relevant_issue(user_query, candidate_issues)

    es_docs = search_elasticsearch(user_query)

    final_answer = generate_final_answer(user_query, kg_match, es_docs)
    return final_answer


In [ ]:
user_query = "My audio is not working in Webex during meetings"
response = full_pipeline(user_query)
print(response)
